In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree
import random
import math

In [2]:
hoteltest = pd.read_csv('hoteltest_testing_set.csv')
hoteltrain = pd.read_csv('hoteltest_training_set.csv')

In [3]:
hoteltest.dtypes
hoteltrain.dtypes

srch_id                          int64
site_id                          int64
visitor_location_country_id      int64
prop_country_id                  int64
prop_id                          int64
prop_starrating                  int64
prop_review_score              float64
prop_brand_bool                  int64
prop_location_score1           float64
prop_location_score2           float64
position                         int64
promotion_flag                   int64
srch_destination_id              int64
srch_length_of_stay              int64
srch_booking_window              int64
srch_adults_count                int64
srch_children_count              int64
srch_room_count                  int64
srch_saturday_night_bool         int64
srch_query_affinity_score       object
orig_destination_distance        int64
random_bool                      int64
click_bool                       int64
booking_bool                     int64
date_time_split                 object
srch_month               

In [4]:
hoteltest.drop(hoteltest.select_dtypes(['object']), inplace=True, axis=1)
hoteltrain.drop(hoteltrain.select_dtypes(['object']), inplace=True, axis=1)

In [5]:
hoteltest_prop_id = hoteltest[['prop_id']]
hoteltest_srch_id = hoteltest[['srch_id']]
hoteltrain_prop_id = hoteltrain[['prop_id']]
hoteltrain_srch_id = hoteltrain[['srch_id']]
hoteltest = hoteltest.drop(columns=['prop_id', 'srch_id'])
hoteltrain = hoteltrain.drop(columns=['prop_id', 'srch_id'])

In [6]:
def getheaders():
    test_headers = list(pd.read_csv(test, nrows = 0).columns)
    train_headers = list(pd.read_csv(train, nrows = 0).columns)
    if os.path.isfile('properties.txt'):
        with open('properties.txt', "r") as f:
            properties = f.read().splitlines()
        check = [item for item in list(set().union(test_headers,train_headers)) if item not in properties]
        if len(check) != 0:
            raise Exception("lists are not the same")
    else:
        properties = list(set().union(test_headers,train_headers))
        file = open('properties.txt', "w")
        for item in properties:
            print(item, file = file)
        file.close()
    return properties

In [7]:
def get_letor_row(df, prop_id, srch_id, entry):
    target = prop_id.iloc[entry][0]
    qid = "qid:" + str(srch_id.iloc[entry][0])
    features = []
    for i, feature in enumerate(properties):
        try: 
            value = df.loc[entry, feature]
        except:
            pass
        else:
            features.append(str(i + 1) + ":" + str(value))
    features = " ".join(features)
    return " ".join([str(target),qid,features])

In [8]:
def open_set(hotel_set_file):
    hotel_set = pd.read_csv(hotel_set_file)
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    hotel_set_prop_id = hotel_set[['prop_id']]
    hotel_set_srch_id = hotel_set[['srch_id']]
    hotel_set.drop(hotel_set.select_dtypes(['object']), inplace=True, axis=1)
    hotel_set_type = hotel_set_file.split('_')[1][:-3]
    file = open("../LambdaMart/%s.txt" % hotel_set_type, "w")
    for entry in range(len(hotel_set)):
        print(get_letor_row(hotel_set, hotel_set_prop_id, hotel_set_srch_id, entry), file=file)
    file.close()

In [9]:
test = 'hoteltest_testing_set.csv'
train = 'hoteltest_training_set.csv'

In [10]:
properties = getheaders()

site_id                          int64
visitor_location_country_id      int64
prop_country_id                  int64
prop_starrating                  int64
prop_review_score              float64
prop_brand_bool                  int64
prop_location_score1           float64
prop_location_score2           float64
price_usd                      float64
promotion_flag                   int64
srch_destination_id              int64
srch_length_of_stay              int64
srch_booking_window              int64
srch_adults_count                int64
srch_children_count              int64
srch_room_count                  int64
srch_saturday_night_bool         int64
orig_destination_distance        int64
random_bool                      int64
srch_month                       int64
srch_year                        int64
booking_month                    int64
vis_star                         int64
vis_usd                          int64
prop_hist_price                  int64
cat_price_usd            